# Python Script: Clean and Export Data for Jetski Business
# Bookings, Revenue, and Ad Data Integration


In [76]:

import pandas as pd

# Load revenue data
revenue_raw = pd.read_csv("revenue_by_day.csv", header=1)

# Rename columns explicitly
revenue_raw.columns = [
    "Date", "Bookings", "Guests", "Subtotal", "Tax",
    "Total", "Total_Paid", "Processing_Fees", "Total_Revenue"
]

# Clean date and revenue values
revenue_raw['Date'] = pd.to_datetime(revenue_raw['Date'], errors='coerce')
revenue_raw['Total_Revenue'] = revenue_raw['Total_Revenue'].replace(r'[$,]', '', regex=True).astype(float)

# Extract clean revenue data
daily_revenue = revenue_raw[['Date', 'Total_Revenue']].dropna()

In [72]:
# Load and clean ad spend data
ads = pd.read_csv("ad_data.csv")

ads.head()

ads['Starts'] = pd.to_datetime(ads['Starts'], errors='coerce')
ads['Ends'] = pd.to_datetime(ads['Ends'], errors='coerce')
ads['Amount Spent (USD)'] = pd.to_numeric(ads['Amount spent (USD)'], errors='coerce')
ads = ads.dropna(subset=['Starts', 'Ends', 'Amount spent (USD)'])

# Expand ads into daily rows
daily_spend = []
for _, row in ads.iterrows():
    duration = (row['Ends'] - row['Starts']).days + 1
    spend_per_day = row['Amount Spent (USD)'] / duration if duration > 0 else 0
    for single_date in pd.date_range(start=row['Starts'], end=row['Ends']):
        daily_spend.append({"Date": single_date, "Spend_Per_Day": spend_per_day})

ad_spend = pd.DataFrame(daily_spend)
daily_ad_spend = ad_spend.groupby("Date")["Spend_Per_Day"].sum().reset_index()

In [77]:
# Merge and calculate ROAS
merged = pd.merge(daily_ad_spend, daily_revenue, on="Date", how="outer")
merged['Spend_Per_Day'] = merged['Spend_Per_Day'].fillna(0)
merged['Total_Revenue'] = merged['Total_Revenue'].fillna(0)
merged['ROAS'] = merged.apply(
    lambda x: x['Total_Revenue'] / x['Spend_Per_Day'] if x['Spend_Per_Day'] > 0 else 0,
    axis=1
)

In [78]:
# Total metrics
spent_days = merged[merged['Spend_Per_Day'] > 0]
total_ad_spend = spent_days['Spend_Per_Day'].sum()
total_revenue = spent_days['Total_Revenue'].sum()
total_roas = total_revenue / total_ad_spend if total_ad_spend > 0 else 0

In [75]:
revenue_raw.to_csv("cleaned_daily_revenue.csv", index=False)
daily_ad_spend.to_csv("cleaned_daily_ad_spend.csv", index=False)
merged.to_csv("merged_revenue_adspend.csv", index=False)
spent_days.to_csv("spent_days.csv", index=False)